# Creating a mind map before getting into action

So after coming across from the mistakes before this notebook, let's plan what all things we need to do so that we perform the crucial steps needed for the NMT to make with all the proper components and the preprocessed data.

**Data Preprocessing Actions**:
1. Shift the target texts.
2. Pad the text with proper annotations.

**Components of the model**:
1. Create *text vectors* of both source and target language separately using TensorFlow `TextVectorization` layer.
2. Create *embedding layers* of both source and target language separately using TensorFlow `Embedding` layer.
3. Create a class known *Encoder* which inherits the properties of TensorFlow class `Layer`. In the sub-class method of creating a layer, we are going to develop the encoder architecture of the NMT. (The architecture will be discussed while we are developing the layer.)
4. Create a class known as *Decoder* which inherits the properties of TensorFlow class `Layer`. In this sub-class method of creating a layer, we are going to develop the decoder architecture of the NMT. (The architecture will be discussed while we are developing the layer.)
5. After creating all the components, we are going to create a class called *EncoderDecoder* which will inherit from TensorFlow class `Model`. In this class we will assemble all of the layers that we have prepared in the above steps and then create a custom call function which will allow us to train the decoder layer as we expect it to do.
6. Create an instance of the *EncoderDecoder* class with all the parameters passed in its constructor and compile the model with `tf.keras.losses.SparseCategoricalCrossentropy()` as loss function and `tf.keras.optimizers.RMSprop()` as the optimizer.
7. We will then create the *train_dataset* and *valid_dataset* using TensorFlow `tf.data` API for better performance of the model training.
8. We will fit the model with the training data with 5 epochs and *callbacks* such as `tf.keras.callbacks.ModelCheckPoint`, `tf.keras.callbacks.EarlyStopping`, `tf.keras.callbacks.ReduceLROnPlateau` and the validation dataset.
9. After training the model we will evaluate the model using different metrics system and predict with an unseen data and observe the quality of the prediction.

# Importing all the necessary libraries

In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import tarfile
import random

# Start the project

As all machine learning project has two phases, this is project is no exception for it. We will first work on the *Data Preprocessing* and then *Model Development*

## Data Preprocessing

In [2]:
# Creating a constant which contains the starting path of the project so that
START_PATH = str(os.getcwd()) + '/'
COMP_DATA_PATH = os.path.join(START_PATH, 'wiki-titles.tgz')
DATA_URL = 'https://www.statmt.org/wmt14/wiki-titles.tgz'
DATA_DIR = os.path.join(START_PATH, 'wiki/hi-en/wiki-titles.hi-en')

### Loading Data

In [3]:
if not os.path.exists(COMP_DATA_PATH):
    print(f'Downloading data from {DATA_URL}')
    !wget https://www.statmt.org/wmt14/wiki-titles.tgz
else:
    print("Data already downloaded")

--2024-04-05 01:53:21--  https://www.statmt.org/wmt14/wiki-titles.tgz
Resolving www.statmt.org (www.statmt.org)... 129.215.32.28
Connecting to www.statmt.org (www.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8168057 (7.8M) [application/x-gzip]
Saving to: ‘wiki-titles.tgz’

wiki-titles.tgz     100%[===================>]   7.79M   308KB/s    in 14s     

2024-04-05 01:53:38 (555 KB/s) - ‘wiki-titles.tgz’ saved [8168057/8168057]



In [4]:
if not os.path.exists(COMP_DATA_PATH):
    print("File does not exist")
else:
    print("The file exists")

The file exists


In [5]:
with tarfile.open(COMP_DATA_PATH, 'r') as tar_ref:
    tar_ref.extractall()
    print("File extracted")

File extracted


### Extracting data from file 

In [7]:
# Extracting the lines from the file of the dataset
with open(DATA_DIR, 'r') as f:
    lines = f.readlines()

['० जनवरी ||| January 0\n',
 '० मार्च ||| March 0\n',
 '१००० ||| 1000\n',
 '१००१ ||| 1001\n',
 '१००२ ||| 1002\n']

In [14]:
# Splitting the data into two list of source language and target language
src_senteces = [line.split('|||')[1][1:-1] for line in lines]
trg_sentences = [line.split('|||')[0][:-1] for line in lines]
src_senteces[:10], trg_sentences[:10], len(src_senteces), len(trg_sentences)

(['January 0',
  'March 0',
  '1000',
  '1001',
  '1002',
  '1003',
  '1004',
  '1005',
  '1006',
  '1007'],
 ['० जनवरी',
  '० मार्च',
  '१०००',
  '१००१',
  '१००२',
  '१००३',
  '१००४',
  '१००५',
  '१००६',
  '१००७'],
 32863,
 32863)

### Visualise the data

In [36]:
def visualise_random_sentences(src_sent, trg_sent):    
    random_idx = random.randint(0, len(src_sent))
    
    print(f"Source sentence: {src_sent[random_idx]}")
    print(f"Target sentence: {trg_sent[random_idx]}")
    
visualise_random_sentences(src_sent= src_senteces,
                    trg_sent= trg_sentences)

Source sentence: 217 BC
Target sentence: २१७ ईसा पूर्व


### Shift the target data with one token

In [43]:
trg_sentences_preprocessed = ['<SOS> ' + sentence + ' <EOS>' for sentence in trg_sentences]
visualise_random_sentences(src_sent= src_senteces,
                           trg_sent= trg_sentences_preprocessed)

Source sentence: Arsenic
Target sentence: <SOS> आर्सेनिक <EOS>


In [45]:
# Calculate the max length of the text for each language lists
src_word_per_sentence = [len(line.split()) for line in src_senteces]
trg_word_per_sentence = [len(line.split()) for line in trg_sentences_preprocessed]

max_src_len = max(src_word_per_sentence)
max_trg_len = max(trg_word_per_sentence)

print(f"Max source sentence length: {max_src_len}")
print(f"Max target sentence length: {max_trg_len}")

Max source sentence length: 13
Max target sentence length: 17


>**Note**: Padding of the data will be done after vectorising the text data

## Model development

We have completed the data preprocessing of the data and are ready to develop the model. Let's discuss the steps we are going to take for it:
1. Create individual components
2. Assemble the model
3. Create Data Pipeline
4. Fit the model
5. Evaluate the model
6. Predict using the model